Park 1: This notebook will be mainly used for the capstone project for the IBM DATA SCIENCE PROFESSIONAL CERTIFICATE on Coursera

In [112]:
import pandas as pd
import numpy as np

In [113]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


Part 2: I will be scraping the Wikipedia Page "List of postal codes of Canada: M"
to create a pandas dataframe of Canada's Postal code, Borough and Neighborhoods.

In [114]:
from bs4 import BeautifulSoup

In [115]:
import requests

Scrape the text of the wikipedia page that includes the table we want

In [116]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [117]:
soup = BeautifulSoup(source, 'lxml')

In [118]:
#print(soup.prettify())

Extract the table out of the text

In [119]:
table = soup.find_all('table')[0]
#print(table.prettify())

Convert into a dataframe

In [120]:
df = pd.read_html(str(table))

In [121]:
df = df[0]
df.head()

,0,1,2
0,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


Clean the Datafram

In [122]:
new_header = df.iloc[0] #grab the first row for the header
df = df[1:] #take the data less the header row
df.columns = new_header #set the header row as the df header

In [123]:
#Delete Row with Borough = Not Assigned
df = df[df.Borough != 'Not assigned']
df.head()

,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


In [126]:
# Combine Rows with same Postcode/Borough
df2 = df.groupby(['Postcode','Borough'],as_index=False, sort=False).agg(', '.join)
df2['Neighbourhood']=set(df2['Neighbourhood'])


Just in case...

In [127]:
#Replacing Not assigned neighborhood with Borough name
df2["Neighbourhood"].replace('Not assigned', df.Borough, inplace=True)

In [128]:
df2.reset_index()
df2.head(12)

,Postcode,Borough,Neighbourhood
0,M3A,North York,"Guildwood, Morningside, West Hill"
1,M4A,North York,"Rouge, Malvern"
2,M5A,Downtown Toronto,St. James Town
3,M6A,North York,"Parkdale, Roncesvalles"
4,M7A,Queen's Park,Leaside
5,M9A,Etobicoke,Humewood-Cedarvale
6,M1B,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M3B,North York,"Humber Bay Shores, Mimico South, New Toronto"
8,M4B,East York,"Bloordale Gardens, Eringate, Markland Wood, Ol..."
9,M5B,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo..."


Look at the shape of the dataframe

In [129]:
df2.shape

(103, 3)

We will get the latitude and longitude coordinates of the neighborhoods

In [145]:
!wget -q -O "toronto_coordinates.csv" http://cocl.us/Geospatial_data
print('Coordinates downloaded!')
coors = pd.read_csv('toronto_coordinates.csv')

Coordinates downloaded!


In [146]:
print(coors.shape)
coors.head()

(103, 3)


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


We will now merge the two dataframes

In [149]:
df_temp = df2.set_index('Postcode')
coors_temp = coors.set_index('Postal Code')
df_coors = pd.concat([df_temp, coors_temp], axis=1, join='inner')

In [150]:
df_coors.index.name = 'PostalCode'
df_coors.reset_index(inplace=True)

In [151]:
print(df_coors.shape)
df_coors.head()

(103, 5)


,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,"Guildwood, Morningside, West Hill",43.753259,-79.329656
1,M4A,North York,"Rouge, Malvern",43.725882,-79.315572
2,M5A,Downtown Toronto,St. James Town,43.654260,-79.360636
3,M6A,North York,"Parkdale, Roncesvalles",43.718518,-79.464763
4,M7A,Queen's Park,Leaside,43.662301,-79.389494
